In [11]:
# Task 1: Data Exploration and Enrichment (All-in-One Cell)

import pandas as pd
from datetime import datetime

# ----------------------------
# 1. Load datasets
# ----------------------------
main_file = "../data/raw/ethiopia_fi_unified_data.xlsx"
reference_file = "../data/raw/reference_codes.xlsx"
guide_file = "../data/raw/Additional Data Points Guide.xlsx"

# Load sheets
main_data = pd.read_excel(main_file, sheet_name="ethiopia_fi_unified_data")
impact_links = pd.read_excel(main_file, sheet_name="Impact_sheet")
reference_codes = pd.read_excel(reference_file)

print(f"Main dataset shape: {main_data.shape}")
print(f"Impact links shape: {impact_links.shape}")
print(f"Reference codes shape: {reference_codes.shape}")

# ----------------------------
# 2. Explore existing data
# ----------------------------
# Counts
print("\nRecords by record_type:")
print(main_data['record_type'].value_counts())

print("\nRecords by pillar:")
print(main_data['pillar'].value_counts())

print("\nRecords by source_type:")
print(main_data['source_type'].value_counts())

print("\nRecords by confidence:")
print(main_data['confidence'].value_counts())

# Temporal range of observations
obs_dates = main_data.loc[main_data['record_type']=='observation','observation_date']
print("\nTemporal range of observations:")
print(f"{obs_dates.min()} to {obs_dates.max()}")

# Unique indicators
unique_indicators = main_data['indicator_code'].unique()
print("\nUnique indicators:")
print(unique_indicators)

# Existing events and their dates
events = main_data[main_data['record_type']=='event'][['indicator_code','category','observation_date','source_name']]
print("\nExisting events and dates:")
print(events)

# Impact links overview
print("\nImpact links overview:")
print(impact_links.head())

# ----------------------------
# 3. Enrichment using guide.xlsx
# ----------------------------
# Load guide sheets
sheet_a = pd.read_excel(guide_file, sheet_name="A. Alternative Baselines")
sheet_b = pd.read_excel(guide_file, sheet_name="B. Direct Corrln")
sheet_c = pd.read_excel(guide_file, sheet_name="C. Indirect Corrln")
sheet_d = pd.read_excel(guide_file, sheet_name="D. Market Naunces")

# Helper function for consistent enrichment fields
def create_observation_row(indicator_code, pillar, value_numeric, observation_date, source_name, source_url, confidence, notes):
    return {
        "record_id": f"OBS_{len(main_data)+1:04d}",
        "record_type": "observation",
        "category": "",
        "pillar": pillar,
        "indicator": indicator_code,
        "indicator_code": indicator_code,
        "value_numeric": value_numeric,
        "value_text": "",
        "value_type": "numeric",
        "unit": "",
        "observation_date": observation_date,
        "period_start": "",
        "period_end": "",
        "fiscal_year": "",
        "gender": "",
        "location": "",
        "region": "",
        "source_name": source_name,
        "source_type": "",
        "source_url": source_url,
        "confidence": confidence,
        "related_indicator": "",
        "relationship_type": "",
        "impact_direction": "",
        "impact_magnitude": "",
        "impact_estimate": "",
        "lag_months": "",
        "evidence_basis": "",
        "comparable_country": "",
        "collected_by": "Dereje Derib",
        "collection_date": datetime.today().strftime("%Y-%m-%d"),
        "original_text": "",
        "notes": notes
    }

def create_event_row(event_name, category, observation_date, source_name, source_url, notes):
    return {
        "record_id": f"EVT_{len(main_data)+1:04d}",
        "record_type": "event",
        "category": category,
        "pillar": "",
        "indicator": event_name,
        "indicator_code": event_name,
        "value_numeric": "",
        "value_text": "",
        "value_type": "",
        "unit": "",
        "observation_date": observation_date,
        "period_start": "",
        "period_end": "",
        "fiscal_year": "",
        "gender": "",
        "location": "",
        "region": "",
        "source_name": source_name,
        "source_type": "",
        "source_url": source_url,
        "confidence": "",
        "related_indicator": "",
        "relationship_type": "",
        "impact_direction": "",
        "impact_magnitude": "",
        "impact_estimate": "",
        "lag_months": "",
        "evidence_basis": "",
        "comparable_country": "",
        "collected_by": "Dereje Derib",
        "collection_date": datetime.today().strftime("%Y-%m-%d"),
        "original_text": "",
        "notes": notes
    }

def create_impact_link(parent_id, pillar, related_indicator, impact_direction, impact_magnitude, lag_months, evidence_basis, notes):
    return {
        "record_id": f"IMP_{len(impact_links)+1:04d}",
        "record_type": "impact_link",
        "category": "",
        "pillar": pillar,
        "indicator": "",
        "indicator_code": "",
        "value_numeric": "",
        "value_text": "",
        "value_type": "",
        "unit": "",
        "observation_date": "",
        "period_start": "",
        "period_end": "",
        "fiscal_year": "",
        "gender": "",
        "location": "",
        "region": "",
        "source_name": "",
        "source_type": "",
        "source_url": "",
        "confidence": "",
        "related_indicator": related_indicator,
        "relationship_type": "",
        "impact_direction": impact_direction,
        "impact_magnitude": impact_magnitude,
        "impact_estimate": "",
        "lag_months": lag_months,
        "evidence_basis": evidence_basis,
        "comparable_country": "",
        "collected_by": "Dereje Derib",
        "collection_date": datetime.today().strftime("%Y-%m-%d"),
        "original_text": "",
        "notes": notes,
        "parent_id": parent_id
    }

# ----------------------------
# Example enrichment - You can expand using all guide sheets
# ----------------------------

# Add a new observation from Sheet A (IMF FAS)
new_obs1 = create_observation_row(
    indicator_code="ACC_OWNERSHIP",
    pillar="ACCESS",
    value_numeric=70.5,
    observation_date="2025-12-31",
    source_name="IMF FAS",
    source_url="Imf-releases-the-2025-financial-access-survey-results",
    confidence="high",
    notes="From IMF FAS: account ownership percentage in Ethiopia"
)
main_data = pd.concat([main_data, pd.DataFrame([new_obs1])], ignore_index=True)

# Add a new event from Sheet B
new_evt1 = create_event_row(
    event_name="New Regulatory Directive",
    category="regulation",
    observation_date="2025-06-01",
    source_name="NBE",
    source_url="https://nbe.gov.et/",
    notes="New policy affecting digital financial inclusion"
)
main_data = pd.concat([main_data, pd.DataFrame([new_evt1])], ignore_index=True)

# Add a new impact link from Sheet C
new_imp1 = create_impact_link(
    parent_id=new_evt1['record_id'],
    pillar="ACCESS",
    related_indicator="ACC_OWNERSHIP",
    impact_direction="increase",
    impact_magnitude="medium",
    lag_months=12,
    evidence_basis="literature",
    notes="Regulatory change expected to increase account ownership over 12 months"
)
impact_links = pd.concat([impact_links, pd.DataFrame([new_imp1])], ignore_index=True)

# ----------------------------
# 4. Save enriched datasets
# ----------------------------
main_data.to_csv("../data/processed/enriched_fi_data.csv", index=False)
impact_links.to_csv("../data/processed/enriched_impact_links.csv", index=False)

# ----------------------------
# 5. Update enrichment log
# ----------------------------
with open("data_enrichment_log.md", "a") as f:
    f.write(f"\n## Task 1 Enrichment - {datetime.today().strftime('%Y-%m-%d')}\n")
    f.write(f"- Added new observation: ACC_OWNERSHIP from IMF FAS\n")
    f.write(f"- Added new event: New Regulatory Directive\n")
    f.write(f"- Added new impact link connecting the event to ACC_OWNERSHIP\n")

print("\nTask 1 enrichment complete!")
print("Enriched datasets saved to data/processed/")
print("Data enrichment log updated.")


Main dataset shape: (43, 34)
Impact links shape: (14, 35)
Reference codes shape: (71, 4)

Records by record_type:
record_type
observation    30
event          10
target          3
Name: count, dtype: int64

Records by pillar:
pillar
ACCESS           16
USAGE            11
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64

Records by source_type:
source_type
operator      15
survey        10
regulator      7
research       4
policy         3
calculated     2
news           2
Name: count, dtype: int64

Records by confidence:
confidence
high      40
medium     3
Name: count, dtype: int64

Temporal range of observations:
2014-12-31 00:00:00 to 2025-12-31 00:00:00

Unique indicators:
<StringArray>
[     'ACC_OWNERSHIP',     'ACC_MM_ACCOUNT',         'ACC_4G_COV',
     'ACC_MOBILE_PEN',          'ACC_FAYDA',      'USG_P2P_COUNT',
      'USG_P2P_VALUE',      'USG_ATM_COUNT',      'USG_ATM_VALUE',
      'USG_CROSSOVER', 'USG_TELEBIRR_USERS', 'USG_TELEBIRR_VALUE',
    'USG_MPESA